### GEP identifiying true solutions

Define GEP

$$
A c = \omega B c
$$



To rewrite as a standard EP one would like to do:
$$
B^{-1} A c = \omega B^{-1} B U c\\
B^{-1} A c = \omega I c
$$

however, this requires that $B$ be invertible ($B$ is a non-singular matrix $==$ has non-zero determinant)

Particle-hole EOM GEP has an indefinite $B$ matrix (symmetric with negative and positive eigenvalues), and, for a single slater determinant reference wfn, $B$ is singular, therefore, we can not directly use the method above.

Approaches for solving the singula EV problem:

* thin SVD

\begin{align*}
X_{(P \times P)} &= U \Sigma V^T\\
&= \sum^{r}_{i=1} \sigma_i u_i v^T_i + \sum^{P}_{i=r+1} 0 * u_i v^T_i\\
&\approx \sum^{r}_{i=1} \sigma_i u_i v^T_i
\end{align*}

* Tikhonov regularization (aka. ridge regresion, weight decay, regularized least-squares)
$$
X = U \Sigma V^T\\
$$
$$
X^T X + \lambda I =  V \Sigma U^T  U \Sigma V^T  + \lambda V V^T\\
X^T X + \lambda I =  V (\Sigma^2 + \lambda I) V^T
$$

$V$, $V^T$ and $ (\Sigma^2 + \lambda I)$ are invertible matrices, their product is too and therefore also $X^T X + \lambda I$.

$\lambda \geq 0$, therefore $(\Sigma^2 + \lambda I) \ge 0$

\begin{align*}
X b &= \omega&\\
X^T X b &= X^T \omega&\\
b &= (X^T X)^{-1} X^T \omega&\\
b &\approx (X^T X + \lambda I)^{-1} X^T \omega&
\end{align*}


Tests:

* check $|B|=0$


* Try diferent approaches to deal with $B$'singular values:

    * Make zero inverse of $B$'s singular values (Current method in EOM package): `Usvd`

    * add small factor to $B$: : `Ushift`

    * Remove eigenvectors in $B$'s null space (expand $A$ in new vector space, solve and transform EOM eigenvectors back): `Utrunc`

    * Try appliying regularization: `Uridge`

\begin{align*}
A c &= \omega B c\\
B^T A c &= \omega B^T B c\\
B^T A c &\approx \omega (B^T B + \lambda I) c
\end{align*}


Scripts used to do tests/generate data (not included):

    test_pheom_energy_consistency.py

In [3]:
# Import modules
import numpy as np
import csv
import matplotlib as mpl
from matplotlib import pyplot

from prettytable import from_csv

In [13]:
def parse_csv(pfile):
    content = list(csv.DictReader(open(pfile, 'r',newline='')))
    Bond = [float(row['Bond']) for row in content]
    epyscf = [float(row['PySCF']) for row in content]
    eUsvd = [float(row['Usvd']) for row in content]
    eUtrunc = [float(row['Utrunc']) for row in content]
    eUshift = [float(row['Ushift']) for row in content]
    eUridge = [float(row['Uridge']) for row in content]
    return Bond, epyscf, eUsvd, eUtrunc, eUshift, eUridge


def plot_disoc(bonds, eref, default, utrunc, ushift, uridge, system, basis, ref='uhf', xunits='Bohr', lims=None):
    reflabel = {'uhf': 'UHF', 'hf': 'RHF'}
    # pyplot.rc('font', family='serif')
    pyplot.rc('xtick', labelsize='x-small')
    pyplot.rc('ytick', labelsize='x-small')
    # Create figure and add axis
    fig = pyplot.figure(figsize=(4, 3))
    ax = fig.add_subplot(1, 1, 1)
    # Add plots
    ax.plot(bonds, eref, marker='o', color="blue", label="PySCF")
    ax.plot(bonds, default, marker='o', color="black", label="U-SVD")
    ax.plot(bonds, utrunc, marker='*', color="green", label="U-trunc")
    ax.plot(bonds, ushift, marker='*', color="black", label="U-shift")
    ax.plot(bonds, uridge, marker='x', color="brown", label="U-ridge")

    # Set the x axis
    ax.set_xlabel(f'Bond distance {xunits}')
    # Set the y axis label of the current axis.
    ax.set_ylabel('Energy (Ha)')
    # Set a title of the current axes.
    ax.set_title(f'{system} disociation, {basis}, ref {reflabel[ref]}')
    # show a legend on the plot
    ax.legend(bbox_to_anchor=(1, 1), loc=1, frameon=False, fontsize=10)
    # Edit the major and minor ticks of the x and y axes
    ax.xaxis.set_tick_params(which='major', size=7, width=1, direction='in', top='on')
    ax.xaxis.set_tick_params(which='minor', size=4, width=1, direction='in', top='on')
    ax.yaxis.set_tick_params(which='major', size=7, width=1, direction='out', right='on')
    ax.yaxis.set_tick_params(which='minor', size=4, width=1, direction='out', right='on')
    # Set the axis limits
    if lims is not None:
        ax.set_xlim(lims['x'][0], lims['x'][1])
        ax.set_ylim(lims['y'][0], lims['y'][1])
    # Edit the major and minor tick locations
    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(0.5))
    ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.1))
    ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.05))
    # ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(2))
    # Display a figure.
    pyplot.show()

Test 1
======

Repeat 10 times de solution to the GEVP using the procedures listed above to transform it into a standard EVP. From the solution eigenvectors the 2-RDM is reconstructed and the ground state energy evaluated; this is the final magniture used to make comparisons. On each iteration the same Hamiltonian matrix (from UHF) is used by all methods.

The results in the table bellow show that Usvd, Utrunc and Uridge are equivalent and tend to be closer to the result from PySCF RPA. However from one iteration to the next, their solutions change.

The methods where either the left-hand-side or both sides are shifted by a small factor (Ushift and AUshift) deviate significantly from the other three (around 0.3-1.0 a.u.), however they give more stable results (they are more robust?)


In [9]:

csvf = "test_pheom_energy_consistency_h2_6-31G(d,p)_b_1.4.csv"
with open(csvf, "r") as fp: 
    x = from_csv(fp, delimiter=',')
    
print(x)

+------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+
| Iter |        PySCF        |         Usvd        |        Utrunc       |        Ushift       |        Uridge       |       AUshift       |
+------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+
|  1   | -1.2121730614490231 |  -1.310005698336214 | -1.3100056983362107 | -1.5933260675444103 | -1.3100056983362056 | -1.9731272485455096 |
|  2   | -1.2121730614490231 | -1.2608506224987677 |  -1.260850622498757 | -1.6130743489694603 | -1.2608506224988063 | -1.9731337789065981 |
|  3   | -1.2121730614490231 | -1.1980610142521904 | -1.1980610142521906 | -1.6111426487699356 | -1.1980610142521906 | -1.9731328926451148 |
|  4   | -1.2121730614490231 | -1.3745707837873318 | -1.3745707837873358 |   -1.592134272639   | -1.3745707837873418 |  -1.973133798398506 |
|  5   | -1.2

Test 1
======

The next graph of $H_2$ disociation path aims to show that the equivalence between the procedures Usvd, Utrunc and Uridge is independent of the geometry chosen for the molecule.

In [10]:
csvf = "test_pheom_energy_stability_h2_cc-pvdz_PES.csv"
molec = 'H2'
basis = '6-31G(d,p)'
xy_lims = {'x': [1.0, 5.0], 'y':[-1.35, -0.95]}

In [11]:
# Parse data file in csv frmat
bonds, epyscf, eUsvd, eUtrunc, eUshift, eUridge = parse_csv(csvf)

In [ ]:
# Disociation path of H2 from diferent GEVP solver procedures
plot_disoc(bonds, epyscf, eUsvd, eUtrunc, eUshift, eUridge, molec, basis, ref='uhf', xunits='Bohr', lims=None)

Test 3
======

Take, for example, the standard GEVP solver in the EOM package, and using the same Hamiltonian matrix as input to the EOM equations, solve two times.

In this case the same answer is obtained every time.

See function `compare_phrpa_instances(atom, bond, basis, spin, unit="Bohr", mode=None)` in test_pheom_energy_consistency.py

Conclusion:

These tests lead to think that the observed energy instability is not due to singularities on the left-hand-sie matrix of the GEVP, but due to changes in the input parameters, in particular the Hamiltonian matrix elements.